# ElasticSearch

In this notebook we have created a search engine using ElasticSearch. 

For our own reference:
* Literature: <https://www.elastic.co/guide/en/elasticsearch/guide/current/index.html>
* Telegraaf XML documents: http://data.politicalmashup.nl/arjan/telegraaf/



Create a search engine for the telegraaf newspaper collection using eg ElasticSearch. Make facets for years and document types. Pay attention to telephone numbers (in mini advertisements). Hieronder een voorbeeld van 1 document (= 1 artikeltje).
Je ziet dat er zelfs een link naar de bron tekst (als plaatje) instaat. De URL linked door naar de nieuwe url http://www.delpher.nl/nl/kranten/view?identifier=ddd%3A010563762%3Ampeg21%3Aa0005&coll=ddd ElasticSearch gebruikt een JSON formaat als invoer, en dit is dus triviaal om te zetten naar JSON.

Each of the following points must be addressed. Create a seperate page on the wiki for each point. Make sure these pages can be found from the menu of your wiki. Explain what you did, and exemplify with links to screenshots/a working system.

* Search as we know it from Google. Give a result page (SERP), with links to the documents and some description of each hit.
* Advanced search. Let a user be able to search in several fields, also in several fields simulteanously. Queries like "return kamervragen by Wilders about XXX with an answer about YYY in the period ZZZ" should be possible. (For the "Telegraaf" collectie, let the user search in both title and tekst fields)
* Do one of the following:
    1. Represent the hits of a query with a wordcloud of 25-50 informative words. The wordcloud should somehow summarise what the collection has to say about the query. You may think of these words as words that you could add to the query in order to improve recall (blind relevance feedback/query expansion).
    2. Represent each document (a kamervraag) with a word-cloud. Also make word-clouds for the question and for the answer. EXAMPLE: The html files in http://data.politicalmashup.nl/arjan/odeii/data_as_html/ contain such wordcloud summaries, which work rather well.   

You can use several techniques to get rid of high frequency, but meaningless words: of course IDF, but also mutual information (see 13.5.1), or of course the technique from the paper by Kaptein et al on wordclouds.

* Give next to a traditional list of results, a timeline in which you indicate how many hits there are over time.
* Give next to the traditional list of results, a table with the number of hits for each political party. Link the party names, which should result in only selecting the hits "ingediend" by members of that party. (Faceted Search) (For the "Telegraaf" collectie, use the dc:subject element as facet values.)
* Evaluate your results Let 2 persons assess the relevancy of the top 10 documents for 5 different queries. Compute Cohen's kappa. Determine the average precision at 10 for your system based on these 10 queries, and the two relevance assesments. Also plot the P@10 (for both judges) for each query, showing differences in hard and easy queries. Describe clearly how you solved differences in judgements. 
Create your queries in the following format:

                    <topic number="6"  >
          <query>kcs</query>
          <description>Find information on the Kansas City Southern railroad.
          </description>
           
        </topic>

        <topic number="16"  >
          <query>arizona game and fish</query>
          <description>I'm looking for information about fishing and hunting
          in Arizona.
          </description>
           
        </topic>
                

So, both provide the actual query, and a description of the information need that was behind the query.
Give a small set of clear guidelines for judging the results, and let your judges follow these guidelines.
It is far more interesting to have difficult queries (both for the search engine and for the judges) than to have queries on which all ten retrieved documents are relevant. So, try to create a good list of information needs.

* Change the ranking of your system, compute the average precision at 10 using your 10 queries, compare the results to your old system, and EXPLAIN what is going on.


# The Search Engine

Before running ES run: 

    export ES_HEAP_SIZE=Half_RAM

where Half_RAM is half your ram

AND: 
in /config/elasticsaerch.yml add 
indices.memory.index_buffer_size: 50% 
(Still need to check if this makes a difference)

To start the Elastic searh serive, please run the following code in commandline:

    ./elasticsearch-2.4.1/bin/elasticsearch --node.name telegraaf

## Initiate connection to the Elastic Search engine

In [2]:
import sys
import json
from elasticsearch import Elasticsearch, helpers

HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST],retry_on_timeout=True)

# If code runs, the connection is made

## Generator to read Telegraaf XML and add them to the ES database

A generator makes it possible to immediately put the XML files/documents into the ES databse

* Remove high frequency, but meaningless words

You can use several techniques to get rid of high frequency, but meaningless words: of course IDF, but also mutual information (see 13.5.1), or of course the technique from the paper by Kaptein et al on wordclouds.

In [3]:
from bs4 import BeautifulSoup
import sys
from os import listdir
from os.path import isfile
import xml.etree.ElementTree as etree
from lxml import etree


def read(year):
    '''
    return a generator for the date, subject(type), 
    title, and text for each item in the given year. 
    '''
    count = 0
    date = ''
    subject =''
    identifier = ''
    text = ''
    title = ''
    for event, elem in etree.iterparse(year,events=("start", "end")):
        if event =='end':
            if elem.tag == '{http://purl.org/dc/elements/1.1/}date':
                date = elem.text
                elem.clear()
            elif elem.tag == '{http://purl.org/dc/elements/1.1/}subject':
                subject = elem.text
                elem.clear()
            elif elem.tag == '{http://purl.org/dc/elements/1.1/}identifier':
                identifier = elem.text
                elem.clear()
            elif elem.tag == 'title':
                if elem.text == None:
                    title = ''
                else:
                    title = elem.text
                elem.clear()
            elif elem.tag == 'p':
                if elem.text == None:
                    text = ''
                else:
                    text = elem.text
                elem.clear()
            elif elem.tag == '{http://www.politicalmashup.nl}root':
                elem.clear()
                yield (date,subject,identifier,title,text)
            else:
                elem.clear()
                
#     soup = BeautifulSoup(open(year,'r'),'xml')
#     for date,subject, title, text, identifier in zip(soup.find_all('date'), soup.find_all('subject'), 
#                                                      soup.find_all('title'), soup.find_all('text'),
#                                                      soup.find_all('identifier')):
#             yield (date.text,subject.text,title.text,text.text,identifier.text)

documents = ['./Telegraaf/'+i for i in listdir('./Telegraaf') if not isfile(i)]

print(documents)

# Create the generator for the bulk importer
# I'm not sure if it's a good idea to use _type here as a subject (which is artcle or advertisement, or more...)
# The score calculation for the Elastic Search database uses whole-index statistics. 
# If you're searching a subsection this will alter the scores! WE WILL NEED TO KEEP THIS IN MIND.
#k = ({'_type':subject, '_index':'telegraaf','_source':{'year':date[:4], 'date':date[5:], 'title':title, 'text':text}} 
#    for year in documents for (date,subject,title,text) in read(year))


['./Telegraaf/telegraaf-1918.xml', './Telegraaf/telegraaf-1922.xml', './Telegraaf/telegraaf-1923.xml', './Telegraaf/telegraaf-1951.xml', './Telegraaf/telegraaf-1961.xml', './Telegraaf/telegraaf-1962.xml', './Telegraaf/telegraaf-1963.xml', './Telegraaf/telegraaf-1965.xml', './Telegraaf/telegraaf-1966.xml', './Telegraaf/telegraaf-1968.xml', './Telegraaf/telegraaf-1969.xml', './Telegraaf/telegraaf-1970.xml', './Telegraaf/telegraaf-1971.xml', './Telegraaf/telegraaf-1972.xml', './Telegraaf/telegraaf-1973.xml', './Telegraaf/telegraaf-1974.xml', './Telegraaf/telegraaf-1975.xml', './Telegraaf/telegraaf-1976.xml', './Telegraaf/telegraaf-1977.xml', './Telegraaf/telegraaf-1978.xml', './Telegraaf/telegraaf-1979.xml', './Telegraaf/telegraaf-1980.xml', './Telegraaf/telegraaf-1981.xml', './Telegraaf/telegraaf-1982.xml', './Telegraaf/telegraaf-1983.xml', './Telegraaf/telegraaf-1984.xml', './Telegraaf/telegraaf-1985.xml', './Telegraaf/telegraaf-1986.xml', './Telegraaf/telegraaf-1987.xml', './Telegraaf/

## Populate ES database

In [39]:
# List of all indices
! curl 'localhost:9200/_cat/indices?v'

health status index                   pri rep docs.count docs.deleted store.size pri.store.size 
yellow open   .marvel-es-1-2016.10.23   1   1       3342            0      2.6mb          2.6mb 
yellow open   .marvel-es-data-1         1   1         26            3       14kb           14kb 
yellow open   .kibana                   1   1          2            0       19kb           19kb 
yellow open   telegraaf                 5   1      13183            0     11.6mb         11.6mb 


In [40]:
# delete any pre-excisting index
es.indices.delete(index='telegraaf', ignore=[404,400])

{u'acknowledged': True}

In [41]:
# Create the telegraaf index in our telegraaf node
es.indices.create(index='telegraaf', ignore=400)

{u'acknowledged': True}

In [42]:
# turn refresh off to speed up bulk import
es.indices.put_settings(index='telegraaf',body={"index" : 
                                            {"refresh_interval" : "30s"
                                            }
                                       })

{u'acknowledged': True}

In [43]:
import time

#Import the information into the database
# bundeling same type together could speed up bulk process
# I
def bulk_per_doc(documents):
    start = time.time()
    print "Starting time:", start
    for doc in documents:
        print "Document:", doc
        failed = 0
        succes = 0
        start_doc = time.time()
        k = ({'_index':'telegraaf', '_type':subject, '_id':identifier, '_source':{'year':date[:4], 
               'date':date[5:], 'title':title, 'text':text}} for (date,subject,identifier,title,text) in read(doc))
        for (ok, res) in helpers.parallel_bulk(es,k, chunk_size=4000,max_chunk_bytes=15728640, request_timeout=10000):
            if not ok:
                failed +=1
            else:
                succes += 1
        end_doc = time.time()
        print '[Done]', end_doc-start_doc
        print "Succesful injections: ", succes
        print "Failed injections:", failed
        print '\n'
    end =time.time()
    print "Finished", (end_doc-start)

bulk_per_doc(documents)    

# It is better to use the per doc populate method, because otherwise the bulk queue will overflow. 
# def bulk_per_doc(documents):
#     start = time.time()
#     print "Starting time:", start

#     for i in documents[:1]:
#         start_doc = time.time()
#         k = ({'_type':subject, '_id':identifier, '_index':'telegraaf','_source':{'year':date[:4], 
#              'date':date[5:], 'title':title, 'text':text}}
#              for (date,subject,title,text,identifier) in read(i))
#         for ok in helpers.parallel_bulk(es,k,chunk_size=500,max_chunk_bytes=15728640):
#             continue
#         end_doc =time.time()
#         print "Finished", (end_doc-start_doc)
#         end = time.time()
#     print "Done:", end - start
        


Starting time: 1477265060.26
Document: ./Telegraaf/telegraaf-1918.xml
[Done] 2.51697707176
Succesful injections:  6263
Failed injections: 0


Document: ./Telegraaf/telegraaf-1922.xml
[Done] 1.60133695602
Succesful injections:  6692
Failed injections: 0


Document: ./Telegraaf/telegraaf-1923.xml
[Done] 0.105178117752
Succesful injections:  228
Failed injections: 0


Document: ./Telegraaf/telegraaf-1951.xml
[Done] 9.98317098618
Succesful injections:  37375
Failed injections: 0


Document: ./Telegraaf/telegraaf-1961.xml
[Done] 32.8531060219
Succesful injections:  106988
Failed injections: 0


Document: ./Telegraaf/telegraaf-1962.xml
[Done] 57.1315381527
Succesful injections:  180677
Failed injections: 0


Document: ./Telegraaf/telegraaf-1963.xml
[Done] 54.427259922
Succesful injections:  120901
Failed injections: 0


Document: ./Telegraaf/telegraaf-1965.xml
[Done] 13.4305539131
Succesful injections:  46140
Failed injections: 0


Document: ./Telegraaf/telegraaf-1966.xml
[Done] 79.994584083

In [38]:
# Set the refresh rate back to default
es.indices.put_settings(index='telegraaf',body={"index" : 
                                            {"refresh_interval" : "1s"
                                            }
                                       })

{u'acknowledged': True}

In [6]:
# Speed improvements/performance: 
# - bootstrap.mlockall: true in config of the file 
# (make sure  ES_HEAP_SIZE is large enough) 
# Parsing whole document xml.cElementTree.parse()
# Streaming the xml document: xml.sax.reader.html

# import xml.etree.ElementTree as etree
# for event, elem in etree.iterparse(xmL, events=('start', 'end', 'start-ns', 'end-ns')):
#  print event, elem
# http://boscoh.com/programming/reading-xml-serially.html
# Event handlers 

## Query system 

* Normalise query
* Get right tokens from the query. Use patterns to split up the query in parts? 
* Put them in the right representation for ES search

In [4]:
def search(query, advanced=False):
    '''
    Given a query it returns a SERP with rakings based on the score
    '''
    if advanced:
        must = []
        if query[0]:
            must.append({"term": {"_type": query[0]}})
        if query[2]:
            must.append({"term": {"year": query[2]}})
        
        q = {"query": 
                {"filtered": 
                    {"query": {
                        "multi_match": 
                            {"query" : query[1],
                             "type" : "cross_fields",  # with 'and' operator 
                             "fields" : ['title', 'text'],
                             "operator" : 'and'
                            }
                        },
                     "filter": 
                        {"bool" : 
                            {"must" : must
                            }
                        }
                    }
                }
            }
        res = es.search(index='telegraaf', size=10, body=q)
        return res
    else:
        # filter_path can help reduce the amount of data that is returned by the es.search
        # The query context is for how well the document fits the query
        # The filter context is a boolean context. Does it match or not.
        # example: Does this timestamp fall into the range 2015 to 2016?
        #
        
        # The outer 'query': is necessary to show that this is the query.  
        q = {'query':
                {'multi_match':
                    {'query' : query,
                     'type' : 'cross_fields',  # with 'and' operator 
                     'fields' : ['title', 'text'],
                     'operator' : 'and'
                     }
                 }
             }
        # In other words, all terms must be present in at least one field for a document to match.
        res = es.search(index='telegraaf', size=10, body=q)
        return res
    

In [5]:
#search('stoomschip engelsche')

## Result page function

* Take query output and use score to order result on a Search Engine Result Page (SERP).
* Return title, link, and description of each hit

-> The description can be a word cloud of 20-25 most informative words. Represent the hits of a query with a wordcloud of 25-50 informative words. The wordcloud should somehow summarise what the collection has to say about the query. You may think of these words as words that you could add to the query in order to improve recall (blind relevance feedback/query expansion). 


Additions
* A timeline with the amount of hits over time
* A table with the number of hits for each political party. Link the party names, which should result in only selecting the hits "ingediend" by members of that party. (Faceted Search) (For the "Telegraaf" collectie, use the dc:subject element as facet values.)

## Advanced Search

The query system will have to be changed to implement this

* Make multiple fields searchable:
    * Title 
    * Tekst
    * Year?
    
Let a user be able to search in several fields, also in several fields simulteanously. Queries like "return kamervragen by Wilders about XXX with an answer about YYY in the period ZZZ" should be possible. (For the "Telegraaf" collectie, let the user search in both title and tekst fields)

In [7]:
# unique_years = list(set( document.date.get_text()[:4]
#                     for document in soup_documents ))

In [8]:
# Determine values for the document type facets
unique_doc_types = list(set(es.indices.get_mapping(index='telegraaf')['telegraaf']['mappings']))

NameError: name 'es' is not defined

In [7]:
import formlayout 

query = formlayout.fedit([('Document type',[0]+unique_doc_types),
               ('Zoektermen',''),
               ('Jaar publicatie','')],
               title="Telegraaf zoekmachine", 
               comment="Wat voor krantenartikel zoek je?")

query[0] = unique_doc_types[query[0]]
# query[2] = unique_years[query[2]]

ImportError: formlayout requires PyQt4, PyQt5 or PySide

## Result page function

* Take query output and use score to order result on a Search Engine Result Page (SERP).
* Return title, link, and description of each hit

### SERP & Worcloud
Since stopwords have high frequencies, they are likely to occupy most places in the word cloud. We therefore remove an extensive stopword list consisting of 571 common English words. Only single words (unigrams) are included in the cloud and stemming is applied. To create a word cloud all terms in the document are sorted by their probabilities and a fixed number of the 25 top ranked terms are kept. The top 10 documents retrieved by a language model run are concatenated and treated as one long document.

In [6]:
from stop_words import get_stop_words
import snowballstemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

def result_page(query, hits, url, n, m):
    """
    Return the SERP information from a given list of
    search results.
    """
    if hits:
        serp = []
        total_text = []
        for i, hit in enumerate(hits[:10]):
            serp.append((None,None))
            serp.append((None, "URL: <a href='%s'>%s</a>" % (url+str(hit['_id']),url+str(hit['_id']))))
            serp.append((None,hit['_source']['title']))
            text = hit['_source']['text'].split()
            total_text = total_text + text
            serp.append((None,"Beschrijving: " + extract_description(query,text,m)))
        #wordcloud = create_wordcloud(text, n)
    else:
        serp = [(None,'\n\n\n\nEr zijn helaas geen zoekresultaten.\n\n\n\n')]
    return serp

def create_wordcloud(text, n):
    """
    Display a wordcloud with at most n words, generated
    from the given text.
    """
    # Filter words to use for the wordcloud, by stemming and stop words removal
    stop_words = get_stop_words("dutch")
    stemmer = snowballstemmer.stemmer("dutch")
    text = [word for word in text if word.lower() not in stop_words]
    text = stemmer.stemWords(text)

    # Plot wordcloud
    wordcloud = WordCloud(max_font_size=40, background_color="white",
                          max_words = n).generate(" ".join(text))
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

def extract_description(query, text, m):
    query = query.split()
    positions = sorted([text.index(word) for word in query 
                        if word in text ])
    
    positions = [i for i in positions if i > 7]

    # If word(s) appeared in text, return these sentences
    if positions:
        description = position_sentences(positions, text, m)
    # If the word only occured in title, return first sentence
    else:
        description = ' '.join(text[0:m]) + '...'
    
    return description

def position_sentences(positions, text, m):
    
    mini = positions[0]
    maxi = mini
    for i in positions[1:]:
        if i > mini and i <= mini + m:
            maxi = i
    diff = int(math.floor((m - (maxi - mini)) / 2))
    return '...'+' '.join(text[mini-diff:maxi+diff])+'...'
    

res = search(query, advanced=True)
serp = result_page(query[1], res['hits']['hits'], "http://kranten.kb.nl/view/article/id/", 50, 15)
results = formlayout.fedit(serp, title="Telegraaf results")

NameError: name 'query' is not defined

## Evaluation

* Manual relevance check
* P@10
* Change the ranking of the system + explain what is going on and why it is improving/decreasing

Evaluate your results Let 2 persons assess the relevancy of the top 10 documents for 5 different queries. Compute Cohen's kappa. Determine the average precision at 10 for your system based on these 10 queries, and the two relevance assesments. Also plot the P@10 (for both judges) for each query, showing differences in hard and easy queries. Describe clearly how you solved differences in judgements. 
So, both provide the actual query, and a description of the information need that was behind the query.
Give a small set of clear guidelines for judging the results, and let your judges follow these guidelines.
It is far more interesting to have difficult queries (both for the search engine and for the judges) than to have queries on which all ten retrieved documents are relevant. So, try to create a good list of information needs.
